# Observational Analysis

This code was run in the UK Biobank Research Analysis Platform to create the observational (not MR) models.

In [ ]:
dx download THEFINALFULLDATASET.csv


In [ ]:
# Install necessary packages
install.packages("lubridate")
library(lubridate)

install.packages('data.table')
library(data.table)

install.packages('timereg')
library(timereg)

install.packages("survival")
library("survival")


In [ ]:
# Getting censoring data aligned
dataset$CensorDate <- fifelse(dataset$REGION == "Scotland", as.Date("2021-07-31"),
                              fifelse(dataset$REGION == "Wales", as.Date("2018-02-28"), as.Date("2021-09-30")))


summary(as.Date(dataset$CensorDate))

# Converting BMI Date to an actual date instead of character
dataset$BMIDate <- as.Date(dataset$BMIDate)

In [ ]:
# Finishing Time Variable based on WHEN things happened
# If they didn't die or experience CVD event, then censor date - first date BMI measured is used
dataset$Time <- ifelse(is.na(dataset$CVDDate) == TRUE & is.na(dataset$Date.of.Death.Inst.0) == TRUE & is.na(dataset$Date.of.Death.Inst.1) == TRUE, dataset$CensorDate - dataset$BMIDate, dataset$Time) 

# Converting time to years
dataset$TimeYear <- dataset$Time/365.24


In [ ]:
# Fitting observational aalen model
FULLMODEL <- aalen(Surv(TimeYear, StatBinary) ~ const(FirstBMI), robust = 0, data = dataset)


summary(FULLMODEL)

In [ ]:
# Creating the SES subcategories
UKBDFNOUNI <- dataset[-grep("College or University degree", dataset$EA), ]
UKBDFUNI <- dataset[grep("College or University degree", dataset$EA), ]
UKBDFLowSES <- dataset[grep("None of the above", dataset$EA), ]
UKBDFHighSES <- dataset[-grep("None of the above", dataset$EA), ]


In [ ]:
NOUNI <- aalen(Surv(TimeYear, StatBinary) ~ const(FirstBMI) + const(AgeBaseline) + const(EverSmoke) + const(AssayType) + const(Biological.Sex) +
                                              const(PC1) + const(PC2) + const(PC3) + const(PC4) + const(PC5) + const(PC6) + const(PC7) + const(PC8) + const(PC9) + const(PC10), robust = 0, data = UKBDFNOUNI))

UNI <- aalen(Surv(TimeYear, StatBinary) ~ const(FirstBMI) + const(AgeBaseline) + const(EverSmoke) + const(AssayType) + const(Biological.Sex) +
                                              const(PC1) + const(PC2) + const(PC3) + const(PC4) + const(PC5) + const(PC6) + const(PC7) + const(PC8) + const(PC9) + const(PC10), robust = 0, data = UKBDFUNI))

LowSES <- aalen(Surv(TimeYear, StatBinary) ~ const(FirstBMI) + const(AgeBaseline) + const(EverSmoke) + const(AssayType) + const(Biological.Sex) +
                                              const(PC1) + const(PC2) + const(PC3) + const(PC4) + const(PC5) + const(PC6) + const(PC7) + const(PC8) + const(PC9) + const(PC10), robust = 0, data = UKBDFLowSES))

HighSES <- aalen(Surv(TimeYear, StatBinary) ~ const(FirstBMI) + const(AgeBaseline) + const(EverSmoke) + const(AssayType) + const(Biological.Sex) +
                                              const(PC1) + const(PC2) + const(PC3) + const(PC4) + const(PC5) + const(PC6) + const(PC7) + const(PC8) + const(PC9) + const(PC10), robust = 0, data = UKBDFHighSES))
